#### In this notebook, let's get the dummies of the categorical columns 

#### Import the libraries

In [1]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.model_selection import cross_val_score

##### Load data

In [2]:
melbourne_file_path = '/Users/alvaro/Kaggle/Machine Learning/2 - Explore the data/melb_data.csv'
melbourne_data = pd.read_csv(melbourne_file_path)

In [3]:
melbourne_data.head()

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,...,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
0,Abbotsford,85 Turner St,2,h,1480000.0,S,Biggin,3/12/2016,2.5,3067.0,...,1.0,1.0,202.0,NaN,NaN,Yarra,-37.7996,144.9984,Northern Metropolitan,4019.0
1,Abbotsford,25 Bloomburg St,2,h,1035000.0,S,Biggin,4/02/2016,2.5,3067.0,...,1.0,0.0,156.0,79.0,1900.0,Yarra,-37.8079,144.9934,Northern Metropolitan,4019.0
2,Abbotsford,5 Charles St,3,h,1465000.0,SP,Biggin,4/03/2017,2.5,3067.0,...,2.0,0.0,134.0,150.0,1900.0,Yarra,-37.8093,144.9944,Northern Metropolitan,4019.0
3,Abbotsford,40 Federation La,3,h,850000.0,PI,Biggin,4/03/2017,2.5,3067.0,...,2.0,1.0,94.0,NaN,NaN,Yarra,-37.7969,144.9969,Northern Metropolitan,4019.0
4,Abbotsford,55a Park St,4,h,1600000.0,VB,Nelson,4/06/2016,2.5,3067.0,...,1.0,2.0,120.0,142.0,2014.0,Yarra,-37.8072,144.9941,Northern Metropolitan,4019.0


##### Drop houses where the target is missing

In [4]:
melbourne_data.dropna(axis=0, subset=['Price'], inplace=True)

##### Prepare our environment

In [5]:
melb_target = melbourne_data.Price
melb_predictors = melbourne_data.drop(['Price'], axis=1)

##### Divide the data into training and test. 

In [6]:
X_train, X_test, y_train, y_test = train_test_split(melb_predictors, 
                                                    melb_target,
                                                    train_size=0.7, 
                                                    test_size=0.3, 
                                                    random_state=0)

##### Missing values (easy model)

In [7]:
cols_with_missing = [col for col in X_train.columns 
                                 if X_train[col].isnull().any()]                                  
candidate_train_predictors = X_train.drop(cols_with_missing, axis=1)
candidate_test_predictors = X_test.drop(cols_with_missing, axis=1)

##### Select Categorical cols

In [8]:
#Select Categorical cols with less than 10 different values per column and numeric cols
low_cardinality_cols = [cname for cname in candidate_train_predictors.columns if 
                                candidate_train_predictors[cname].nunique() < 10 and
                                candidate_train_predictors[cname].dtype == "object"]
numeric_cols = [cname for cname in candidate_train_predictors.columns if 
                                candidate_train_predictors[cname].dtype in ['int64', 'float64']]
my_cols = low_cardinality_cols + numeric_cols
train_predictors = candidate_train_predictors[my_cols]
test_predictors = candidate_test_predictors[my_cols]

##### Get dummies

In [9]:
one_hot_encoded_training_predictors = pd.get_dummies(train_predictors)

In [10]:
one_hot_encoded_training_predictors.T

,4431,775,6528,12949,7475,9619,10518,692,3657,7362,...,5874,4373,7891,9225,4859,13123,3264,9845,10799,2732
Rooms,1.0000,4.0000,3.0000,4.00000,2.0000,3.00000,1.00000,2.0000,5.0000,3.0000,...,2.0000,2.0000,2.0000,4.00000,3.0000,3.00000,3.0000,4.00000,3.00000,4.0000
Distance,5.5000,13.0000,8.0000,23.00000,12.2000,11.40000,5.00000,9.2000,5.6000,2.5000,...,6.1000,2.3000,9.2000,11.40000,4.5000,5.20000,10.5000,6.70000,12.00000,6.4000
Postcode,3070.0000,3204.0000,3016.0000,3136.00000,3147.0000,3204.00000,3182.00000,3104.0000,3101.0000,3121.0000,...,3182.0000,3051.0000,3146.0000,3204.00000,3181.0000,3056.00000,3081.0000,3058.00000,3073.00000,3011.0000
Bedroom2,1.0000,4.0000,3.0000,4.00000,2.0000,3.00000,1.00000,3.0000,5.0000,3.0000,...,2.0000,2.0000,2.0000,4.00000,3.0000,3.00000,3.0000,4.00000,3.00000,4.0000
Bathroom,1.0000,1.0000,1.0000,2.00000,1.0000,2.00000,1.00000,1.0000,2.0000,2.0000,...,1.0000,2.0000,1.0000,2.00000,2.0000,1.00000,1.0000,2.00000,1.00000,2.0000
Landsize,0.0000,580.0000,470.0000,1880.00000,803.0000,220.00000,0.00000,0.0000,534.0000,324.0000,...,0.0000,146.0000,97.0000,568.00000,183.0000,212.00000,748.0000,441.00000,606.00000,319.0000
Lattitude,-37.7783,-37.9263,-37.8587,-37.79325,-37.8683,-37.90958,-37.86305,-37.7938,-37.8044,-37.8278,...,-37.8626,-37.7959,-37.8637,-37.90688,-37.8562,-37.77695,-37.7416,-37.73572,-37.72057,-37.7943
Longtitude,144.9954,145.0422,144.8871,145.27724,145.1064,145.03371,144.99110,145.0666,145.0386,144.9885,...,144.9818,144.9464,145.0632,145.02827,145.0042,144.95785,145.0481,144.97256,145.02615,144.8875
Propertycount,11364.0000,6795.0000,6380.0000,11925.00000,2894.0000,2397.00000,13240.00000,7809.0000,10331.0000,1123.0000,...,13240.0000,6821.0000,10412.0000,2397.00000,7717.0000,11918.00000,2947.0000,11204.00000,21650.00000,7570.0000
Type_h,0.0000,1.0000,1.0000,1.00000,1.0000,0.00000,0.00000,0.0000,1.0000,1.0000,...,0.0000,1.0000,0.0000,1.00000,1.0000,1.00000,1.0000,1.00000,1.00000,1.0000


##### Check the results with categorical values in dummies and without categorical

In [11]:
def get_mae(X, y):
    # multiple by -1 to make positive MAE score instead of neg value returned as sklearn convention
    return -1 * cross_val_score(RandomForestRegressor(50), 
                                X, y, 
                                scoring = 'neg_mean_absolute_error').mean()

predictors_without_categoricals = train_predictors.select_dtypes(exclude=['object'])

mae_without_categoricals = get_mae(predictors_without_categoricals, y_train)

mae_one_hot_encoded = get_mae(one_hot_encoded_training_predictors, y_train)

print('Mean Absolute Error when Dropping Categoricals: ' + str(int(mae_without_categoricals)))
print('Mean Abslute Error with One-Hot Encoding: ' + str(int(mae_one_hot_encoded)))

/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


Mean Absolute Error when Dropping Categoricals: 189756
Mean Abslute Error with One-Hot Encoding: 176678
